## Wikipedia tool

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [4]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [6]:
tool.name

'wikipedia'

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import google.generativeai as genai
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

## Making my custom retriever tool

In [14]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
chunk=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

In [15]:
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='\n\n\n\n\nGet started with LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith\u200bPythonTypeScriptpip install -U langsmithyarn add l

In [16]:
chunk

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content="Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricin

In [18]:
embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectordb=FAISS.from_documents(chunk,embeddings)

I0000 00:00:1723571004.309890 52745389 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [19]:
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x117bd2610>)

In [20]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search","Search for information abt LangSmith. For any questions abt Langsmith, for any questions about Langsmith, you must use this tool")

In [21]:
retriever_tool.name

'langsmith_search'

## Making arxiv tool

In [23]:
# Arxiv

from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv_tool=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name

'arxiv'

In [24]:
tools=[tool,arxiv_tool,retriever_tool]

## __Agents__

In [26]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

I0000 00:00:1723571865.934337 52745389 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1723571865.936173 52745389 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [28]:
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/Users/sam/Desktop/LLMs/venv/lib/python3.11/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [29]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

### Agent Executor

In [30]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwc

In [36]:
agent_executor.invoke({"input":"Tell me about UWB Based obstacle detection by Jamsheed Manja Ppallan"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'UWB Based obstacle detection by Jamsheed Manja Ppallan'}`




/Users/sam/Desktop/LLMs/venv/lib/python3.11/site-packages/langchain_community/utilities/arxiv.py:156: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  ).results()


Published: 2023-11-02
Title: IR-UWB Radar-based Situational Awareness System for Smartphone-Distracted Pedestrians
Authors: Jamsheed Manja Ppallan, Ruchi Pandey, Yellappa Damam, Vijay Narayan Tiwari, Jamsheed Manja Ppallan co-authored a paper titled "IR-UWB Radar-based Situational Awareness System for Smartphone-Distracted Pedestrians" which was published on November 2, 2023. 


> Finished chain.


{'input': 'Tell me about UWB Based obstacle detection by Jamsheed Manja Ppallan',
 'output': 'Jamsheed Manja Ppallan co-authored a paper titled "IR-UWB Radar-based Situational Awareness System for Smartphone-Distracted Pedestrians" which was published on November 2, 2023. \n'}